#2.3 Implement an ELIZA-like program, using substitutions such as those described on page 12. You might want to choose a different domain than a Rogerian psychologist, although keep in mind that you would need a domain in which your program can legitimate engage in a lot of simple repetition.

In [25]:
#This rule-based chatbot is implemented like a fruit seller
import re

#Initialize all the constant
PRICES = {
    "mango": 50,
    "apple": 30,
    "orange": 40,
    "watermelon": 60,
    "pomegrante": 49,
    "durian": 200
}

In [30]:
#Get the input sentence
input_sentence = input("Ask the chatbot something: ").lower()

#Set up all the possible cases (simple)
def tell_the_prices(input_sentence):

  avalible_fruits = list(PRICES.keys())
  plurals = [fruit + "s" for fruit in avalible_fruits]
  avalible_fruits.extend(plurals)
  pattern = r"\b(" + "|".join(avalible_fruits) + r")\b"
  response = ["Okay, I will list the prices of all the fruits you want to buy:\n"]

  fruits = re.findall(pattern, input_sentence)
  for fruit in fruits:
    if fruit[-1] == "s":
      fruit = fruit[:-1]
    response.append(f"-The cost of {fruit} is ${PRICES[fruit]} per kg\n")
  response.append("How do you want to buy?")
  return "".join(response)


def tell_the_total_prices(input_sentence):
  total = 0
  avalible_fruits = list(PRICES.keys())
  plurals = [fruit + "s" for fruit in avalible_fruits]
  avalible_fruits.extend(plurals)

  pattern = r"([0-9])\D*?\b(" + "|".join(avalible_fruits) + r")s?\b"
  amount_fruits = re.findall(pattern, input_sentence)
  response = []

  for i in range(1, len(amount_fruits.groups()), 2):
    amount = amount_fruits.group(i)
    fruit = amount_fruits.group(i + 1)
    if fruit[-1] == "s":
      fruit = fruit[:-1]
    price = PRICES[fruit] * amount
    total += price
    response.append(f"{amount}kg of {fruit} cost ${price}\n")

  response.append(f"The total price is ${total}")
  return "".join(response)

def greet(input_sentence):
  return "Our fruits were harvested on the farm the previous day, which is the largest farms in the city, so they're all fresh and high-quality"

def handling_cases(input_sentence):

  fruits_list = list(PRICES.keys())
  avalible_fruits = [fruit + "s" for fruit in fruits_list]
  if re.search(r"(good|fresh|rotten|sweet|nice)", input_sentence):
    return greet(input_sentence)

  elif re.search(r"(how much|cost|costs|price|prices)", input_sentence):
    return tell_the_prices(input_sentence)

  elif re.search(r"([0-9])\D*?\b(" + "|".join(avalible_fruits) + r")s?\b"):
    return tell_the_total_prices(input_sentence)

response = handling_cases(input_sentence)
print(response)






Ask the chatbot something: are these good
Our fruits were harvested on the farm the previous day, which is the largest farms in the city, so they're all fresh and high-quality


#2.6 Implement a minimum edit distance algorithm and use your hand-computer results to check your code.

In [41]:
DEL_COST = 1
INS_COST = 1
SUBS_COST = 1

def minimum_edit_distance(source, target):
  m = len(source)
  n = len(target)
  dp = [[0] * (n + 1) for _ in range(m + 1)]

  for i in range(m + 1):
    dp[i][0] = i
  for j in range(n + 1):
    dp[0][j] = j

  for i in range(1, m + 1):
    for j in range(1, n + 1):

      if source[i - 1] == target[j - 1]:
        dp[i][j] = dp[i - 1][j - 1]


      else:
        dp[i][j] = min(
            dp[i][j - 1] + INS_COST,
            dp[i - 1][j] + DEL_COST,
            dp[i - 1][j - 1] + SUBS_COST
        )

  min_edit_distance = dp[m][n]
  return min_edit_distance

min_edit_distance= minimum_edit_distance("EXTENTION", "INTENTION")

print(f"Minimum edit distance: {min_edit_distance}")

Minimum edit distance: 2


#2.7 Augment the minimum edit distance algorithm to output an alignment; you will need to store poiunters and add a stage to compute the backtrace

In [42]:
def minimum_edit_distance_with_alignments(source, target):
  m = len(source)
  n = len(target)
  dp = [[0] * (n + 1) for _ in range(m + 1)]
  alignment_matrix = [[[] for i in range(n + 1)] for j in range(m + 1)]

  for i in range(m + 1):
    dp[i][0] = i
  for j in range(n + 1):
    dp[0][j] = j

  for i in range(1, m + 1):
    for j in range(1, n + 1):

      if source[i - 1] == target[j - 1]:
        dp[i][j] = dp[i - 1][j - 1]
        alignment_matrix[i][j].append((i - 1, j - 1))

      else:
        dp[i][j] = min(
            dp[i][j - 1] + INS_COST,
            dp[i - 1][j] + DEL_COST,
            dp[i - 1][j - 1] + SUBS_COST
        )

        if dp[i][j] == dp[i][j - 1] + INS_COST:
          alignment_matrix[i][j].append((i, j - 1))
        if dp[i][j] == dp[i - 1][j] + DEL_COST:
          alignment_matrix[i][j].append((i - 1, j))
        if dp[i][j] == dp[i - 1][j - 1] + SUBS_COST:
          alignment_matrix[i][j].append((i - 1, j - 1))

  align_x = []
  align_y = []
  # Backtracking to find the alignment
  def backtrace(i, j, alignment_matrix, cur_source, cur_target):
    if i == 0 and j == 0:
      align_x.append("".join(cur_source[::-1]))
      align_y.append("".join(cur_target[::-1]))
      return

    for prev_i, prev_j in alignment_matrix[i][j]:

      if prev_i == i - 1 and prev_j == j - 1:
        cur_source.append(source[i - 1])
        cur_target.append(target[j - 1])
        backtrace(prev_i, prev_j, alignment_matrix, cur_source, cur_target)
        cur_source.pop()
        cur_target.pop()

      elif prev_i == i - 1:
        cur_source.append(source[i - 1])
        cur_target.append("+")
        backtrace(prev_i, prev_j, alignment_matrix, cur_source, cur_target)
        cur_source.pop()
        cur_target.pop()


      elif prev_j == j - 1:
        cur_source.append("-")
        cur_target.append(target[j - 1])
        backtrace(prev_i, prev_j, alignment_matrix, cur_source, cur_target)
        cur_source.pop()
        cur_target.pop()


  backtrace(m, n, alignment_matrix, [], [])
  min_edit_distance = dp[m][n]
  return min_edit_distance, align_x, align_y

min_edit_distance, align_x, align_y = minimum_edit_distance_with_alignments("EXTENTION", "INTENTION")
print("Minimum Edit Distance:", min_edit_distance)
print("Alignment:")
for i in range(len(align_x)):
  print(align_x[i])
  print(align_y[i])
  print()

Minimum Edit Distance: 2
Alignment:
EXTENTION
INTENTION

